## Run and predict model on all images

In [1]:
import torch
from efficientnet_pytorch import EfficientNet
import glob
from PIL import Image
import numpy as np
from fungi_classification import get_transforms
from  tqdm import tqdm


data_dir = "/data/AIDatasets/fungi/DF20M/"
best_trained_model = "/home/cin/Projects/FungiClassification/saved_models/DF20M-EfficientNet-B0_best_loss.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_classes = 183
model = EfficientNet.from_name('efficientnet-b0', num_classes=n_classes)
checkpoint = torch.load(best_trained_model)
model.load_state_dict(checkpoint)
model.to(device).eval()


transform = get_transforms("valid")
im_paths = sorted(glob.glob(data_dir + "*.JPG"))
pred_labels = []

for im_path in tqdm(im_paths):
    im = Image.open(im_path)
    im = transform(image=np.asarray(im))
    im = im["image"].to(device).unsqueeze(0)

    with torch.no_grad():
        pred_labels.append(model(im).argmax(1).item())

/home/cin/miniconda3/envs/fungidev/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
 65%|██████▌   | 23763/36393 [10:05<07:55, 26.55it/s]

In [ ]:
import pickle
with open("data_sort_new.npy", "wb") as fp:
    np.savez(fp, im_names=im_paths, pred_labels=pred_labels)

## Move images to new dir

In [ ]:
import os
import shutil

out_dir = "/home/cin/Projects/FungiClassification/saved_models/sort_ims"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
    for i in range(183):
        os.mkdir(os.path.join(out_dir, str(i)))

for im_path, label in zip(im_paths, pred_labels):
    shutil.copyfile(im_path, os.path.join(out_dir, label, os.path.basename(im_path)))


In [12]:
os.path.basename(out_dir+".jpg")

'sort_ims.jpg'